In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
                                     Conv2DTranspose, Concatenate, UpSampling2D, GlobalAveragePooling2D)
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


dataset_dir = "C:\\Users\\Lenovo\\.cache\\kagglehub\\datasets\\uzairkhan45\\breast-cancer-patients-mris\\versions\\1\\Breast Cancer Patients MRI's\\train"
img_size = (128, 128)
batch_size = 32


datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2, 
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)


train_generator_cnn = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

train_generator_unet = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


def build_cnn_model():
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_unet_binary_classification(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)

    
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

   
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

  
    u4 = UpSampling2D((2, 2))(c3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)

    u5 = UpSampling2D((2, 2))(c4)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)


    x = GlobalAveragePooling2D()(c5)  
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


cnn_model = build_cnn_model()
unet_model = build_unet_binary_classification()


cnn_history = cnn_model.fit(train_generator_cnn, validation_data=val_generator, epochs=10)
unet_history = unet_model.fit(train_generator_unet, validation_data=val_generator, epochs=10)


cnn_model.save("cnn_model.h5")
unet_model.save("unet_model.h5")



def ensemble_model(cnn, unet):
    
    cnn_output = cnn.output


    unet_output = unet.output

   
    cnn_output_resized = Dense(64, activation='relu')(cnn_output)
    unet_output_resized = Dense(64, activation='relu')(unet_output)


    merged = tf.keras.layers.concatenate([cnn_output_resized, unet_output_resized])
    output = Dense(1, activation='sigmoid')(merged)

    ensemble_model = Model(inputs=[cnn.input, unet.input], outputs=output)
    ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return ensemble_model


ensemble = ensemble_model(cnn_model, unet_model)
ensemble.summary()

ensemble_history = ensemble.fit(
    [train_generator_cnn, train_generator_unet],
    validation_data=([val_generator, val_generator]),
    epochs=10
)


ensemble.save("ensemble_model.h5")


def evaluate_model(model, generator, title):
    loss, accuracy = model.evaluate(generator)
    print(f"{title} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

    y_true = generator.classes
    y_pred = (model.predict(generator) > 0.5).astype("int32")

    print(f"Confusion Matrix for {title}:")
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))


evaluate_model(cnn_model, val_generator, "CNN Model")
evaluate_model(unet_model, val_generator, "Lightweight U-Net Model")
evaluate_model(ensemble, val_generator, "Ensemble Model")


def plot_history(history, title):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f'{title} Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{title} Loss')
    plt.legend()

    plt.show()

# Plot all models
plot_history(cnn_history, "CNN")
plot_history(unet_history, "U-Net")
plot_history(ensemble_history, "Ensemble")


Found 1120 images belonging to 2 classes.
Found 1120 images belonging to 2 classes.
Found 280 images belonging to 2 classes.
Epoch 1/10
35/35 [==============================] - 18s 490ms/step - loss: 0.7281 - accuracy: 0.5125 - val_loss: 0.6861 - val_accuracy: 0.6750
Epoch 2/10
35/35 [==============================] - 18s 501ms/step - loss: 0.6717 - accuracy: 0.5714 - val_loss: 0.7206 - val_accuracy: 0.6321
Epoch 3/10
35/35 [==============================] - 18s 517ms/step - loss: 0.6345 - accuracy: 0.6134 - val_loss: 0.7946 - val_accuracy: 0.5500
Epoch 4/10
35/35 [==============================] - 17s 494ms/step - loss: 0.6024 - accuracy: 0.6643 - val_loss: 0.8654 - val_accuracy: 0.5607
Epoch 5/10
35/35 [==============================] - 17s 497ms/step - loss: 0.5634 - accuracy: 0.6795 - val_loss: 0.8952 - val_accuracy: 0.6107
Epoch 6/10
35/35 [==============================] - 18s 500ms/step - loss: 0.5279 - accuracy: 0.7241 - val_loss: 1.2567 - val_accuracy: 0.4893
Epoch 7/10
35/35 

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'keras.preprocessing.image.DirectoryIterator'>"}), <class 'NoneType'>

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 128, 128, 64  1792        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_21_input (InputLayer)   [(None, 128, 128, 3  0           []                               
                                )]                                                          

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix


y_true = []
y_pred = []


for i in range(len(val_ensemble_generator)):
    (X_cnn, X_unet), y_batch = val_ensemble_generator[i]
    
    
    preds = ensemble.predict([X_cnn, X_unet])
   
    preds_binary = (preds > 0.5).astype(int)
    
    y_true.extend(y_batch)
    y_pred.extend(preds_binary)


y_true = np.array(y_true)
y_pred = np.array(y_pred)


print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred))


print("\n🔍 Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


loss, accuracy = ensemble.evaluate(val_ensemble_generator)
print(f"\n✅ Model Evaluation:\nLoss: {loss:.4f}\nAccuracy: {accuracy:.4f}")


1/1 [==============================] - 1s 796ms/step

📊 Classification Report:
              precision    recall  f1-score   support

         0.0       0.56      0.79      0.65       140
         1.0       0.64      0.39      0.48       140

    accuracy                           0.59       280
   macro avg       0.60      0.59      0.57       280
weighted avg       0.60      0.59      0.57       280


🔍 Confusion Matrix:
[[110  30]
 [ 86  54]]
9/9 [==============================] - 9s 1s/step - loss: 0.6789 - accuracy: 0.5714

✅ Model Evaluation:
Loss: 0.6789
Accuracy: 0.5714
